In [8]:
loan_data = {
     "C123": {
       "customer_name": "Ravi Kumar",
       "total_due": 8400,
       "due_date": "2025-01-15",
       "dpd": 32
     },
     "C456": {
       "customer_name": "Aditi Sharma",
       "total_due": 15200,
       "due_date": "2025-02-01",
       "dpd": 5
     }
   }

In [9]:
# EXAMPLE CONVERSATION
# Bot: Hello, this is Jiya calling from ABC Finance. Am I speaking with Ravi Kumar?

# User: Yes, speaking.

# Bot: Thank you for confirming, Ravi. Your loan amount of rupees 20000 is pending and overdue for 50 days. When can you make the payment?

# User: I can pay this Friday.

# Bot: Thank you, Ravi. I will note that. Please ensure payment is done on Friday

In [10]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("groq:llama-3.1-8b-instant")


In [25]:
from langchain.tools import tool

@tool
def get_customer_details(customer_id: str) -> str:
    """
    Fetches user details for a customer ID and name from JSON data.
    The chatbot should ask like "Hello, this is Jiya calling from ABC Finance. Am I speaking with {customer_name}?"
    """
    if customer_id in loan_data:
        return loan_data[customer_id]
    else:
        return "Customer ID not found."


# @tool 
# def get_confirmation(query: str, response: str) -> str:
#     """
#     Fetches the confirmation from the user. If the response is yes then proceed to fetch the loan details.
#     """
#     return "User has confirmed their identity."

# @tool
# def get_loan_details(customer_id: str, customer_name: str) -> str:
#     f"""
#     If the response is yes from {get_confirmation} then fetch the loan details.
#     Now if the response is yes then fetch the due amount. For example: total_due": 8400,
#     The chatbot should ask like "Thank you for confirming, {customer_name}. Your loan amount of rupees {total_due} is pending from {due_date}. When can you make the payment?
#     """
#     if customer_id in loan_data and loan_data[customer_id]["customer_name"] == customer_name:
#         return loan_data[customer_id]["total_due"]
#     else:
#         return "No due amount found."


@tool
def get_loan_details(customer_id: str, customer_name: str) -> str:
    """
    Fetches user confirmation for a customer ID and name from JSON data. 
    If the user confirms their identity by yes or related word, then  fetch the due amount. For example: total_due": 8400,
    The chatbot should ask like "Thank you for confirming, {customer_name}. Your loan amount of rupees {total_due} is pending from {due_date}. When can you make the payment?
    """
    if customer_id in loan_data and loan_data[customer_id]["customer_name"] == customer_name:
        return loan_data[customer_id]["total_due"]
    else:
        return "No due amount found."

@tool
def record_commitment(customer_id: str, commitment_date: str):
    """Records the user's payment commitment date. And Say Thank you for confirming, {customer_name}. I will note that. Please ensure payment is done on {commitment_date} or {commitment_day}"""
    return "User has committed to pay on {commitment_date}"



In [26]:
from langchain.agents import create_agent

prompt = """
You are Jiya from ABC Finance. You MUST follow these conversation rules:

1. Step 1: Ask if you are speaking with the correct customer (use get_customer_name).
2. Step 2: Wait for user to say “yes” before using get_loan_details.
3. Step 3: After confirmation, fetch loan details using get_loan_details.
4. Step 4: Tell them the due amount and ask when they can make the payment.
5. Step 5: When user provides a date, call record_commitment.
6. Chat must be polite and compliant.

Follow steps strictly.
"""
tools = [get_customer_details, get_loan_details, record_commitment]
agent = create_agent(llm, tools=tools, system_prompt=prompt)



In [13]:
query = "What is the total due for customer C123?"
for event in agent.stream(
    {"messages": [{"role": "user", "content": query}]},
    stream_mode="values",
):
    event["messages"][-1].pretty_print()

================================ Human Message =================================

What is the total due for customer C123?
================================== Ai Message ==================================
Tool Calls:
  get_customer_details (59bvyet3a)
 Call ID: 59bvyet3a
  Args:
    customer_id: C123
  get_loan_details (qhkhhhm8n)
 Call ID: qhkhhhm8n
  Args:
    customer_id: C123
    customer_name: John Doe
================================= Tool Message =================================
Name: get_loan_details

No due amount found.
================================== Ai Message ==================================

Hello, this is Jiya calling from ABC Finance. Am I speaking with Ravi Kumar?


In [14]:
from langchain_core.messages import HumanMessage

# Keep running conversation history here
chat_history = []


def chat(user_text: str):
    """Send a message to Jiya and keep memory across turns."""
    global chat_history

    # Add latest user message to history
    chat_history.append(HumanMessage(content=user_text))

    # Run the agent with full history as state
    state = agent.invoke({"messages": chat_history})

    # Agent returns updated messages (including tool calls + AI reply)
    chat_history = state["messages"]

    # Last message is the latest AI reply
    bot_msg = chat_history[-1]
    print("Jiya:", bot_msg.content)


In [20]:
def start_call(customer_id: str):
    """Start an outbound call: first message is from Jiya, based on customer_id."""
    global chat_history

    # Reset conversation for a fresh call
    chat_history = []

    # Internal instruction to the agent – not shown to the real user
    init_message = (
        f"Start an outbound collection call for customer id {customer_id}. "
        "First, look up their details using the tools if needed, then say: "
        "'Hello, this is Jiya calling from ABC Finance. Am I speaking with <customer_name>?'"
    )

    # Kick off the agent with this synthetic 'user' instruction
    from langchain_core.messages import HumanMessage
    chat_history.append(HumanMessage(content=init_message))

    state = agent.invoke({"messages": chat_history})
    chat_history = state["messages"]

    bot_msg = chat_history[-1]
    print("Jiya:", bot_msg.content)


In [22]:
start_call("C456")

Jiya: Hello, this is Jiya calling from ABC Finance. Am I speaking with Aditi Sharma?


In [23]:
chat("Yes, speaking.")          # confirmation


Jiya: Thank you for confirming, Aditi Sharma. Your loan amount of rupees 15200 is pending from 2025-02-01. When can you make the payment?


In [24]:
chat("I can pay tomorrow.")  # commitment

Jiya: Thank you for confirming, Aditi Sharma. I will note that. Please ensure payment is done on 2025-02-03 or 2nd day.
